In [1]:
import os
import pandas

# 1. Carregamento dos dados

Vamos começar nosso pipeline lendo o conjunto de dados fornecidos. Vamos aproveitar para ver o que tipo de informação esse conjunto de dados tem.

In [2]:
dataset_path = os.getenv("DATASET_PATH")
full_dataset_df = pandas.read_csv(dataset_path)
full_dataset_df

,product_id,seller_id,query,search_page,position,title,concatenated_tags,creation_date,price,weight,express_delivery,minimum_quantity,view_counts,order_counts,category
0,11394449,8324141,espirito santo,2,6,Mandala Espírito Santo,mandala mdf,2015-11-14 19:42:12,171.890000,1200.0,1,4,244,NaN,Decoração
1,15534262,6939286,cartao de visita,2,0,Cartão de Visita,cartao visita panfletos tag adesivos copos lon...,2018-04-04 20:55:07,77.670000,8.0,1,5,124,NaN,Papel e Cia
2,16153119,9835835,expositor de esmaltes,1,38,Organizador expositor p/ 70 esmaltes,expositor,2018-10-13 20:57:07,73.920006,2709.0,1,1,59,NaN,Outros
3,15877252,8071206,medidas lencol para berco americano,1,6,Jogo de Lençol Berço Estampado,t jogo lencol menino lencol berco,2017-02-27 13:26:03,118.770004,0.0,1,1,180,1.0,Bebê
4,15917108,7200773,adesivo box banheiro,3,38,ADESIVO BOX DE BANHEIRO,adesivo box banheiro,2017-05-09 13:18:38,191.810000,507.0,1,6,34,NaN,Decoração
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38502,16561714,9813770,nossa senhora de perolas,1,7,Imagem Nossa Senhora Aparecida em Perolas 25cm,senhora aparecida bebe perolas,2018-04-02 19:43:07,71.200000,706.0,1,4,315,15.0,Decoração
38503,12716324,6483096,lembrancinha personalizada dia dos pais,1,27,LEMBRANCINHA - DIA DOS PAIS,lembrancinhas,2018-07-10 11:41:08,14.650000,7.0,1,26,1288,17.0,Lembrancinhas
38504,972256,4840665,copo metalizado,1,3,Copos Metalizados - Rosé,despedida bianca metalizados xvdakaw lembranci...,2018-02-17 15:53:23,11.580000,25.0,1,104,306,NaN,Lembrancinhas
38505,7291315,6420917,festa do pijama menino,1,36,Kit Festa do Pijama Meninos,festa pijama,2016-08-21 22:33:59,131.330000,0.0,1,11,55,NaN,Lembrancinhas


Para entender melhor o problema, vamos ver quais são as categorias desse conjunto de dados, e qual a distribuição dessas categorias.

In [3]:
full_dataset_df["category"].unique()
full_dataset_df.groupby("category").product_id.count()

category
Bebê                   7026
Bijuterias e Jóias      951
Decoração              8846
Lembrancinhas         17759
Outros                 1148
Papel e Cia            2777
Name: product_id, dtype: int64

- parece que não precisamos fazer nenhuma modificação nos rótulos, não existem versões duplicadas com mudanças pequenas.
- todas categorias parecem ter um número razoável de amostras.

# 2. Transformação de dados

Vamos separar nossos dados em conjunto de treino, teste e validação.

In [4]:
def split_train_test_validate(full_df):
    shuffled_dataset_df = full_df.sample(frac=1).reset_index(drop=True)
    train_cut_idx = int(len(full_df) * .6)
    validation_cut_idx = train_cut_idx + int(len(full_df) * .2)
    return (
        full_df[:train_cut_idx],
        full_df[train_cut_idx:validation_cut_idx],
        full_df[validation_cut_idx:]
    )

train_data, test_data, validate_data = split_train_test_validate(full_dataset_df)
assert(len(full_dataset_df) == len(train_data) + len(test_data) + len(validate_data))

Agora vamos definir o dado que será input no nosso modelo. Como já comentamos antes, não parece ser necessário fazer nenhuma modificação nos rótulos (`category`), mas para o conjunto de características vamos precisar fazer algumas mudanças.

Por conhecimento prévio, e olhando a [descrição de cada coluna do conjunto de dados](https://github.com/elo7/data7_oss/tree/master/elo7-search), parece fazer sentido utilizar os campos de texto: `title`, com o título do produto, `concatenated_tags`, que são tags concatenadas, escolhidas pelo usuário, e `query`, que é uma busca que levou um usuário ao produto.

In [5]:
def feature_extractor(dataframe):
    concatenated_text_information = dataframe["title"] + " " + dataframe["query"] + " " + dataframe["concatenated_tags"].fillna("")
    return concatenated_text_information

assert(feature_extractor(train_data[:1])[0] == "Mandala Espírito Santo espirito santo mandala mdf")

Antes de criar nosso classificador, ainda precisamos transformar nossa feature em um vetor de valores númericos.

Decidimos fazer essa transformação porque pretendemos usar um classificador SVM. Além disso, essa transformação parece ser recomendada para aprendizado de máquina em documentos de texto (veja: [Working With Text Data](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)).

Para fazer essa transformação, vamos usar o conjunto de textos concatenados como nosso corpus, e, a partir de uma ordanação arbitrária de palavras, construir vetores caracteríscos que indicam na i-ésima posição, a frequência da i-ésima palavra.

Obs: também testamos usar TF-IDF, mas a melhora no erro de teste foi menor que 1%, então decidimos usar apenas a frequência de palavras (TF) porque é uma transformação mais simples.

Antes de procurar por parâmetros, vamos testar que esse conjunto de características faz sentido, já treinando um modelo simples de SVM.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm
from sklearn.pipeline import Pipeline
import numpy as np


classifier_pipeline = Pipeline([
    ("word_counter", CountVectorizer()),
    ("tfidf", TfidfTransformer()),
    ("classifier", svm.SVC(kernel="linear"))
])

classifier_pipeline.fit(feature_extractor(train_data), train_data["category"])
test_predict = classifier_pipeline.predict(feature_extractor(test_data))
np.mean(test_predict == test_data["category"])

0.8917023763147643

Conseguimos 89% de acurácia no conjunto de teste com esse modelo. Parece bom, agora vamos só refinar nossa escolha olhando para alguns parâmetros do modelo.

# 3. Modelagem

Vamos usar um classificador da classe Support Vector Machine (SVM). Usaremos um kernel linear, por simplicidade; fizemos um teste com kernel RBF no experimento acima, mas não houve uma melhora expressiva, então preferimos um kernel linear, por ser mais simples.

Para selecionar o parâmetro de regularização C, vamos rodar um teste de validação cruzada com os dados de treinamento apenas. Além disso, vamos também testar se na etapa de transformação usaremos apenas TF (term frequency) ou TF-IDF (term frequency inverse document frequency).

In [8]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

parameters = {"classifier__C": [1, 10], "tfidf__use_idf": [True, False]}
cv_classifier = GridSearchCV(classifier_pipeline, parameters)
cv_classifier.fit(feature_extractor(train_data), train_data["category"])

print(cv_classifier.cv_results_)

{'mean_fit_time': array([20.14665813, 18.21661448, 17.37038579, 16.02546186]), 'std_fit_time': array([0.51016763, 0.07791894, 0.4784684 , 0.10845874]), 'mean_score_time': array([4.0139698 , 3.67437077, 3.40702238, 3.16996284]), 'std_score_time': array([0.35115098, 0.04957153, 0.16921197, 0.12102844]), 'param_classifier__C': masked_array(data=[1, 1, 10, 10],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'param_tfidf__use_idf': masked_array(data=[True, False, True, False],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'classifier__C': 1, 'tfidf__use_idf': True}, {'classifier__C': 1, 'tfidf__use_idf': False}, {'classifier__C': 10, 'tfidf__use_idf': True}, {'classifier__C': 10, 'tfidf__use_idf': False}], 'split0_test_score': array([0.8844406 , 0.88032893, 0.87859771, 0.88595542]), 'split1_test_score': array([0.88985068, 0.88400779, 0.8907163 , 0.89179831]), 'split2_test_score

Olhando o `mean_test_score`, podemos ver que os resultados para as combinações de parâmetros ficaram parecidas, mas com vantagem dos parâmetros `C = 1` e `tfidf = False`.  

In [9]:
# best estimator
chosen_classifier = cv_classifier.best_estimator_

# test error
import numpy as np

test_feature_tf = feature_extractor(test_data)
predicted = chosen_classifier.predict(test_feature_tf)
np.mean(predicted == test_data["category"])

0.8923516426438125

# 4. Validação do modelo

Agora que já escolhemos nosso modelo, vamos validá-lo

In [10]:
validate_feature_tf = feature_extractor(validate_data)
predicted = chosen_classifier.predict(validate_feature_tf)
np.mean(predicted == validate_data["category"])

0.8962607115035056

Nosso modelo escolhido **acertou a categoria de 89% dos produtos** do conjunto de validação.

# 5. Exportação do modelo
Vamos exportar o pipeline treinado do nosso modelo

In [19]:
# baby_class = model_wrapper({
#     "query": "carrinho de bebe",
#     "title": "carrinho de bebe",
#     "concatenated_tags": "bebe"
# })
# assert(baby_class == "Bebê")

Agora, vamos criar um pickle para salvar nossa função que embala o classificador.

In [20]:
import pickle


model_file = os.getenv("MODEL_PATH")
with open(model_file, "wb") as f:
    pickle.dump(chosen_classifier, f)

In [21]:
model_file

'/usr/src/data/classify_function.pickle'